# Generate minimals (-1)

In [ ]:
import numpy as np
import torch
import math
import matplotlib.pyplot as plt
from torch.autograd import Variable
import torch.nn as nn
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import pickle
from scipy.io import loadmat
import os
import matplotlib.cm as cm

dn=21
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device to train")

mycase="find_minimals"
casenum="m1_k008"


all_path="/content/drive/MyDrive/Colab_Notebooks/2023_mep/0120/"
model_Parameters_name = all_path+casenum+"/"+"model_MEP_"+mycase[0:2]+"_"+"pre2"+"pretrain"+".pth" # Path

model_name=all_path+casenum+"/"+"model_MEP_"+mycase+"_"+casenum+".pth"

alpha1=10000
alpha5=1
kappa=0.08


load_model = False
batches = 20000

learning_rate = 1e-4

dn=31
x_dim=2

pre_train_step=500


def mkdir_pre(path):
  folder = os.path.exists(path)
  if not folder:
    os.makedirs(path)
    print("---  new folder...  ---")
    print("---  OK  ---")
  else:
    print("---  There is this folder!  ---")


def fig_loss_batch_pre(plt_batch,loss_batch):
    plt.figure(figsize=(5,4))
    plt.semilogy(plt_batch,loss_batch,'b-')
    plt.xlabel("Batches")
    plt.ylabel("Loss")

def fig_cos_pre(plt_batch,lg_batch,j):
    fig = plt.figure(figsize=(5,4))
    plt.semilogy(plt_batch,lg_batch,'b-')
    plt.xlabel("Batches")
    plt.ylabel("loss on grid")
    if j == batches-500:
      plt.savefig(all_path+casenum+"/"+"lgloss"+".jpg")


def fig_countour_pre(xx,phi,j):
    xx1=xx[:,0]
    xx2=xx[:,1]

    xx1=xx1.reshape(dn,dn)
    xx2=xx2.reshape(dn,dn)

    phi_plot_sq=phi.reshape(dn,dn)


    fig2 = plt.figure(figsize=(24,4))
    fig2.add_subplot(1,1,1)
    picori=phi_plot_sq.reshape(dn,dn)
    pic=np.zeros((dn,dn,3))

    B=(np.sign(picori)+1)/2*picori
    C=(np.sign(picori)-1)/2*picori


    pic[:,:,0]=B
    pic[:,:,-1]=C

    plt.imshow(pic)


    plt.savefig(all_path+casenum+"/"+str(j)+".jpg")



class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_tanh_stack = nn.Sequential(
            nn.Linear(2,32),
            nn.Tanh(),
            nn.Linear(32, 32),
            nn.Tanh(),
            nn.Linear(32, 32),
            nn.Tanh(),
            nn.Linear(32, 32),
            nn.Tanh(),
            nn.Linear(32, 1)
        )

    def forward(self, s):
        phi_pred = self.linear_tanh_stack(s)
        return phi_pred



def train(model):
    loss_batch=[]
    plt_batch=[]
    lg_batch=[]

    for i in range(batches):

        if i<1000:
            learning_rate=1e-3
        elif i<10000:
            learning_rate=1e-4
        else:
            learning_rate=1e-5

        if i<2000:
          alpha1=10
          alpha5=1
        else:
          alpha1=10
          alpha5=1

        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

        datanum=500
        x_np = np.random.uniform(0.001,0.999,(datanum,2))


        x_tensor=Variable(torch.from_numpy(x_np),requires_grad=True).to(device)

        x_all=[]
        for j in range(x_dim):
          x_all.append(x_tensor[:,j:j+1])

        x_tensor_all=torch.cat(x_all,1)
        phi_int_tensor=model(x_tensor_all)

        if i<pre_train_step:
          loss_res=torch.mean((phi_int_tensor+1)**2)
          bound_samp=100

          xlu=np.random.uniform(0.001,0.999,(bound_samp,2))
          xlu[:,0]=0.0
          xlu_tensor=Variable(torch.from_numpy(xlu),requires_grad=True).to(device)


          xru=np.random.uniform(0.001,0.999,(bound_samp,2))
          xru[:,0]=1.0
          xru_tensor=Variable(torch.from_numpy(xru),requires_grad=True).to(device)


          xut=np.random.uniform(0.001,0.999,(bound_samp,2))
          xut[:,1]=1.0
          xut_tensor=Variable(torch.from_numpy(xut),requires_grad=True).to(device)

          xub=np.random.uniform(0.001,0.999,(bound_samp,2))
          xub[:,1]=0.0
          xub_tensor=Variable(torch.from_numpy(xub),requires_grad=True).to(device)

          phi_lu=model(xlu_tensor)
          phi_ru=model(xru_tensor)
          phi_ut=model(xut_tensor)
          phi_ub=model(xub_tensor)

          phi_p1_np=1.0*np.ones((bound_samp))
          phi_m1_np=-1.0*np.ones((bound_samp))

          phi_p1=Variable(torch.from_numpy(phi_p1_np),requires_grad=True).to(device)
          phi_m1=Variable(torch.from_numpy(phi_m1_np),requires_grad=True).to(device)

          loss_bound=torch.mean((phi_lu.reshape(-1)-phi_p1)**2)+torch.mean((phi_ru.reshape(-1)-phi_p1)**2)+torch.mean((phi_ut.reshape(-1)-phi_m1)**2)+torch.mean((phi_ub.reshape(-1)-phi_m1)**2)

          loss=alpha1*loss_res+alpha5*loss_bound

        else:
          gradx_all=[]
          for j in range(x_dim):
            gradx_int = torch.autograd.grad(outputs=phi_int_tensor,inputs=x_all[j],grad_outputs=torch.ones_like(x_tensor[:,j:j+1]),create_graph=True)[0]
            gradx_all.append(gradx_int)

          gradx2_int=0.0
          for j in range(x_dim):
            gradx2_j=torch.autograd.grad(outputs=gradx_all[j],inputs=x_all[j],grad_outputs=torch.ones_like(x_tensor[:,j:j+1]),create_graph=True)[0]
            gradx2_int=gradx2_int+gradx2_j


          loss_res=torch.mean((kappa**2*gradx2_int-(phi_int_tensor**3-phi_int_tensor))**2)

          bound_samp=100

          xlu=np.random.uniform(0.00001,0.99999,(bound_samp,2))
          xlu[:,0]=0.0
          xlu_tensor=Variable(torch.from_numpy(xlu),requires_grad=True).to(device)


          xru=np.random.uniform(0.00001,0.99999,(bound_samp,2))
          xru[:,0]=1.0
          xru_tensor=Variable(torch.from_numpy(xru),requires_grad=True).to(device)


          xut=np.random.uniform(0.00001,0.99999,(bound_samp,2))
          xut[:,1]=1.0
          xut_tensor=Variable(torch.from_numpy(xut),requires_grad=True).to(device)

          xub=np.random.uniform(0.00001,0.99999,(bound_samp,2))
          xub[:,1]=0.0
          xub_tensor=Variable(torch.from_numpy(xub),requires_grad=True).to(device)

          phi_lu=model(xlu_tensor)
          phi_ru=model(xru_tensor)
          phi_ut=model(xut_tensor)
          phi_ub=model(xub_tensor)

          phi_p1_np=1.0*np.ones((bound_samp))
          phi_m1_np=-1.0*np.ones((bound_samp))

          phi_p1=Variable(torch.from_numpy(phi_p1_np),requires_grad=True).to(device)
          phi_m1=Variable(torch.from_numpy(phi_m1_np),requires_grad=True).to(device)

          loss_bound=torch.mean((phi_lu.reshape(-1)-phi_p1)**2)+torch.mean((phi_ru.reshape(-1)-phi_p1)**2)+torch.mean((phi_ut.reshape(-1)-phi_m1)**2)+torch.mean((phi_ub.reshape(-1)-phi_m1)**2)
          loss = alpha1*loss_res +alpha5*loss_bound

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i) % 50 == 0:
            x1 = np.linspace(0.001,0.999,dn,endpoint=True)
            x2 = np.linspace(0.001,0.999,dn,endpoint=True)

            xx1,xx2=np.meshgrid(x1,x2)
            xx1=xx1.reshape((-1,1))
            xx2=xx2.reshape((-1,1))

            xx=np.hstack((xx1,xx2))

            xx_tensor=Variable(torch.from_numpy(xx),requires_grad=True).to(device)

            xx_all=[]
            for j in range(x_dim):
              xx_all.append(xx_tensor[:,j:j+1])

            xx_int_tensor=torch.cat(xx_all,1)

            phi=model(xx_int_tensor)

            gradxx_all=[]
            abs_gradxx_phi_2=0.0
            for j in range(x_dim):
              gradxx_int = torch.autograd.grad(outputs=phi,inputs=xx_all[j],grad_outputs=torch.ones_like(xx_all[j]),create_graph=True)[0]
              gradxx_all.append(gradxx_int)

            gradxx2_int=0.0
            for j in range(x_dim):
              gradxx2_j=torch.autograd.grad(outputs=gradxx_all[j],inputs=xx_all[j],grad_outputs=torch.ones_like(xx_all[j]),create_graph=True)[0]
              gradxx2_int=gradxx2_int+gradxx2_j


            L_phi=(kappa**2*gradxx2_int-(phi**3-phi))**2
            L_int=torch.mean(L_phi,0)

            lg_batch.append(L_int.item())

            if i<pre_train_step:
              loss,batch=loss.item(),i
              print("iteration=",i,"loss=", loss)
            else:
              loss, batch = alpha1*loss_res.item()+alpha5*loss_bound.item(), i
              print(f'batches: {batch+1}')
              print(f'loss1: {loss_res} loss_bound: {loss_bound}')

            loss_batch.append(loss)
            plt_batch.append(i+1)
            plt.figure()

        if (i) % 500 == 0:
            fig_cos_pre(plt_batch,lg_batch,i)
            fig_loss_batch_pre(plt_batch,loss_batch)
            fig_countour_pre(xx,phi.cpu().detach().numpy(),i)
            plt.show()


            torch.save(model.state_dict(), model_name)
            print("Saved PyTorch Model State to " +str(model_name))

    return




if __name__=='__main__':

    mkdir_pre(all_path+casenum+"/")
    print(all_path+casenum+"/")


    model = NeuralNetwork().to(device)
    model = model.double()

    #model = train_pre(model)
    train(model)







# Generate minimals (1)

In [ ]:
import numpy as np
import torch
import math
import matplotlib.pyplot as plt
from torch.autograd import Variable
import torch.nn as nn
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import pickle
from scipy.io import loadmat
import os
import matplotlib.cm as cm


device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device to train")

mycase="find_minimals"
casenum="p3_k005"


all_path="/content/drive/MyDrive/Colab_Notebooks/2023_mep/0120/"

model_name=all_path+casenum+"/"+"model_MEP_"+mycase+"_"+casenum+".pth" # Path


alpha1=10000
alpha5=1
kappa=0.05


load_model = False
batches = 10000

learning_rate = 1e-4

dn=31
x_dim=2

pre_train_step=500


def mkdir_pre(path):
  folder = os.path.exists(path)
  if not folder:
    os.makedirs(path)
    print("---  new folder...  ---")
    print("---  OK  ---")
  else:
    print("---  There is this folder!  ---")


def fig_loss_batch_pre(plt_batch,loss_batch):
    plt.figure(figsize=(5,4))
    plt.semilogy(plt_batch,loss_batch,'b-')
    plt.xlabel("Batches")
    plt.ylabel("Loss")

def fig_cos_pre(plt_batch,lg_batch,j):
    fig = plt.figure(figsize=(5,4))
    plt.semilogy(plt_batch,lg_batch,'b-')
    plt.xlabel("Batches")
    plt.ylabel("loss on grid")
    if j == batches-500:
      plt.savefig(all_path+casenum+"/"+"lgloss"+".jpg")


def fig_countour_pre(xx,phi,j):
    xx1=xx[:,0]
    xx2=xx[:,1]

    xx1=xx1.reshape(dn,dn)
    xx2=xx2.reshape(dn,dn)

    phi_plot_sq=phi.reshape(dn,dn)


    fig2 = plt.figure(figsize=(24,4))
    fig2.add_subplot(1,1,1)
    picori=phi_plot_sq.reshape(dn,dn)
    pic=np.zeros((dn,dn,3))

    B=(np.sign(picori)+1)/2*picori
    C=(np.sign(picori)-1)/2*picori


    pic[:,:,0]=B
    pic[:,:,-1]=C

    plt.imshow(pic)


    plt.savefig(all_path+casenum+"/"+str(j)+".jpg")



class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_tanh_stack = nn.Sequential(
            nn.Linear(2,32),
            nn.Tanh(),
            nn.Linear(32, 32),
            nn.Tanh(),
            nn.Linear(32, 32),
            nn.Tanh(),
            nn.Linear(32, 32),
            nn.Tanh(),
            nn.Linear(32, 1)
        )

    def forward(self, s):
        phi_pred = self.linear_tanh_stack(s)
        return phi_pred



def train(model):
    loss_batch=[]
    plt_batch=[]
    lg_batch=[]

    for i in range(batches):
        if i<2000:
            learning_rate=1e-3
        elif i<30000:
            learning_rate=1e-4
        else:
            learning_rate=1e-5

        if i<2000:
          alpha1=10
          alpha5=1
        else:
          alpha1=10
          alpha5=1

        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

        datanum=500
        x_np = np.random.uniform(0.001,0.999,(datanum,2))


        x_tensor=Variable(torch.from_numpy(x_np),requires_grad=True).to(device)

        x_all=[]
        for j in range(x_dim):
          x_all.append(x_tensor[:,j:j+1])

        x_tensor_all=torch.cat(x_all,1)
        phi_int_tensor=model(x_tensor_all)

        if i<pre_train_step:
          loss_res=torch.mean((phi_int_tensor-1)**2)
          bound_samp=100

          xlu=np.random.uniform(0.001,0.999,(bound_samp,2))
          xlu[:,0]=0.0
          xlu_tensor=Variable(torch.from_numpy(xlu),requires_grad=True).to(device)


          xru=np.random.uniform(0.001,0.999,(bound_samp,2))
          xru[:,0]=1.0
          xru_tensor=Variable(torch.from_numpy(xru),requires_grad=True).to(device)


          xut=np.random.uniform(0.001,0.999,(bound_samp,2))
          xut[:,1]=1.0
          xut_tensor=Variable(torch.from_numpy(xut),requires_grad=True).to(device)

          xub=np.random.uniform(0.001,0.999,(bound_samp,2))
          xub[:,1]=0.0
          xub_tensor=Variable(torch.from_numpy(xub),requires_grad=True).to(device)

          phi_lu=model(xlu_tensor)
          phi_ru=model(xru_tensor)
          phi_ut=model(xut_tensor)
          phi_ub=model(xub_tensor)

          phi_p1_np=1.0*np.ones((bound_samp))
          phi_m1_np=-1.0*np.ones((bound_samp))

          phi_p1=Variable(torch.from_numpy(phi_p1_np),requires_grad=True).to(device)
          phi_m1=Variable(torch.from_numpy(phi_m1_np),requires_grad=True).to(device)

          loss_bound=torch.mean((phi_lu.reshape(-1)-phi_p1)**2)+torch.mean((phi_ru.reshape(-1)-phi_p1)**2)+torch.mean((phi_ut.reshape(-1)-phi_m1)**2)+torch.mean((phi_ub.reshape(-1)-phi_m1)**2)

          loss=alpha1*loss_res+alpha5*loss_bound

        else:
          gradx_all=[]
          for j in range(x_dim):
            gradx_int = torch.autograd.grad(outputs=phi_int_tensor,inputs=x_all[j],grad_outputs=torch.ones_like(x_tensor[:,j:j+1]),create_graph=True)[0]
            gradx_all.append(gradx_int)

          gradx2_int=0.0
          for j in range(x_dim):
            gradx2_j=torch.autograd.grad(outputs=gradx_all[j],inputs=x_all[j],grad_outputs=torch.ones_like(x_tensor[:,j:j+1]),create_graph=True)[0]
            gradx2_int=gradx2_int+gradx2_j


          loss_res=torch.mean((kappa**2*gradx2_int-(phi_int_tensor**3-phi_int_tensor))**2)

          bound_samp=100

          xlu=np.random.uniform(0.00001,0.99999,(bound_samp,2))
          xlu[:,0]=0.0
          xlu_tensor=Variable(torch.from_numpy(xlu),requires_grad=True).to(device)


          xru=np.random.uniform(0.00001,0.99999,(bound_samp,2))
          xru[:,0]=1.0
          xru_tensor=Variable(torch.from_numpy(xru),requires_grad=True).to(device)


          xut=np.random.uniform(0.00001,0.99999,(bound_samp,2))
          xut[:,1]=1.0
          xut_tensor=Variable(torch.from_numpy(xut),requires_grad=True).to(device)

          xub=np.random.uniform(0.00001,0.99999,(bound_samp,2))
          xub[:,1]=0.0
          xub_tensor=Variable(torch.from_numpy(xub),requires_grad=True).to(device)


          phi_lu=model(xlu_tensor)
          phi_ru=model(xru_tensor)
          phi_ut=model(xut_tensor)
          phi_ub=model(xub_tensor)

          phi_p1_np=1.0*np.ones((bound_samp))
          phi_m1_np=-1.0*np.ones((bound_samp))

          phi_p1=Variable(torch.from_numpy(phi_p1_np),requires_grad=True).to(device)
          phi_m1=Variable(torch.from_numpy(phi_m1_np),requires_grad=True).to(device)

          loss_bound=torch.mean((phi_lu.reshape(-1)-phi_p1)**2)+torch.mean((phi_ru.reshape(-1)-phi_p1)**2)+torch.mean((phi_ut.reshape(-1)-phi_m1)**2)+torch.mean((phi_ub.reshape(-1)-phi_m1)**2)
          loss = alpha1*loss_res +alpha5*loss_bound

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i) % 50 == 0:
            x1 = np.linspace(0.001,0.999,dn,endpoint=True)
            x2 = np.linspace(0.001,0.999,dn,endpoint=True)

            xx1,xx2=np.meshgrid(x1,x2)
            xx1=xx1.reshape((-1,1))
            xx2=xx2.reshape((-1,1))

            xx=np.hstack((xx1,xx2))

            xx_tensor=Variable(torch.from_numpy(xx),requires_grad=True).to(device)

            xx_all=[]
            for j in range(x_dim):
              xx_all.append(xx_tensor[:,j:j+1])

            xx_int_tensor=torch.cat(xx_all,1)

            phi=model(xx_int_tensor)

            gradxx_all=[]
            abs_gradxx_phi_2=0.0
            for j in range(x_dim):
              gradxx_int = torch.autograd.grad(outputs=phi,inputs=xx_all[j],grad_outputs=torch.ones_like(xx_all[j]),create_graph=True)[0]
              gradxx_all.append(gradxx_int)


            gradxx2_int=0.0
            for j in range(x_dim):
              gradxx2_j=torch.autograd.grad(outputs=gradxx_all[j],inputs=xx_all[j],grad_outputs=torch.ones_like(xx_all[j]),create_graph=True)[0]
              gradxx2_int=gradxx2_int+gradxx2_j


            L_phi=(kappa**2*gradxx2_int-(phi**3-phi))**2
            L_int=torch.mean(L_phi,0)

            lg_batch.append(L_int.item())

            if i<pre_train_step:
              loss,batch=loss.item(),i
              print("iteration=",i,"loss=", loss)
            else:
              loss, batch = alpha1*loss_res.item()+alpha5*loss_bound.item(), i
              print(f'batches: {batch+1}')
              print(f'loss1: {loss_res} loss_bound: {loss_bound}')

            loss_batch.append(loss)
            plt_batch.append(i+1)
            plt.figure()

        if (i) % 500 == 0:
            fig_cos_pre(plt_batch,lg_batch,i)
            fig_loss_batch_pre(plt_batch,loss_batch)
            fig_countour_pre(xx,phi.cpu().detach().numpy(),i)
            plt.show()


            torch.save(model.state_dict(), model_name)
            print("Saved PyTorch Model State to " +str(model_name))

    return




if __name__=='__main__':

    mkdir_pre(all_path+casenum+"/")
    print(all_path+casenum+"/")


    model = NeuralNetwork().to(device)
    model = model.double()

    #model = train_pre(model)
    train(model)







# Main

In [ ]:

import numpy as np
import torch
import math
import matplotlib.pyplot as plt
from torch.autograd import Variable
import torch.nn as nn
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import pickle
from scipy.io import loadmat
import os
import matplotlib.cm as cm


device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device to train")


all_path="/content/drive/MyDrive/Colab_Notebooks/2023_mep/0120/"
path_m1="/content/drive/MyDrive/Colab_Notebooks/2023_mep/0120/m3_k005/model_MEP_find_minimals_m3_k005.pth"
path_p1="/content/drive/MyDrive/Colab_Notebooks/2023_mep/0120/p3_k005/model_MEP_find_minimals_p3_k005.pth"


mycase="ac_2d"
casenum="NBC3_k005_init3_beta10lg_13"

path_lg=all_path+casenum+"/"+"lgloss"+".pkl"

plot_use_all=[]

torch.set_printoptions(precision=10)


model_name= all_path+casenum+"/"+"model_MEP_"+mycase+"_"+casenum+".pth"
model_Parameters_name=model_name


kappa=0.05
load_model = False
batches = 20000
beta = 10
learning_rate = 1e-4
dimension = 1
dn=28
x_dim=2



alpha1=1
alpha4=0
alpha3=0.001
alpha5=1



def mkdir(path):
  folder = os.path.exists(path)
  if not folder:
    os.makedirs(path)
    print("---  new folder...  ---")
    print("---  OK  ---")
  else:
    print("---  There is this folder!  ---")


def fig_loss_batch(plt_batch,loss_batch):
    plt.figure(figsize=(5,4))
    plt.plot(plt_batch,loss_batch,'b-')
    plt.xlabel("Batches")
    plt.ylabel("Loss")

def fig_cos(plt_batch,lg_batch,j):
    fig = plt.figure(figsize=(5,4))
    plt.plot(plt_batch,lg_batch,'b-')
    plt.xlabel("Batches")
    plt.ylabel("$l_g$")
    if j == batches-500:
      plt.savefig(all_path+casenum+"/"+"lgloss"+".jpg")



def fig_countour(xx,phi,ep,j):
    xx1=xx[:,0]
    xx2=xx[:,1]

    xx1=xx1.reshape(dn**2,dn)
    xx2=xx2.reshape(dn**2,dn)

    phi_plot_sq=phi.reshape(dn**2,dn)


    fig1 = plt.figure(figsize=(24,8))
    dn_draw=list(range(0,dn,int(dn/9)))

    count=1


    for i in dn_draw:
      fig1.add_subplot(2,5,count)
      plt.contourf(xx1[:,i].reshape(dn,dn),xx2[:,i].reshape(dn,dn),phi_plot_sq[:,i].reshape(dn,dn))
      plt.title("Energy:"+str(ep[i]))
      count=count+1

    fig2 = plt.figure(figsize=(24,8))
    count=1

    for u in dn_draw:
      fig2.add_subplot(2,5,count)
      picori=phi_plot_sq[:,u].reshape(dn,dn)
      pic=np.zeros((dn,dn,3))

      B=(np.sign(picori)+1)/2*picori
      C=(np.sign(picori)-1)/2*picori

      pic[:,:,0]=B
      pic[:,:,-1]=C

      ee=round(ep[u],3)
      plt.imshow(pic)
      plt.title("Energy:"+str(ee))
      count=count+1


    plt.savefig(all_path+casenum+"/"+str(j)+".pdf")

    fig3 = plt.figure(figsize=(24,8))
    count=1

    for u in dn_draw:
      fig3.add_subplot(2,5,count)
      picori=phi_plot_sq[:,u].reshape(dn,dn)
      pic=np.zeros((dn,dn,3))

      B=(np.sign(picori)+1)/2*picori
      C=(np.sign(picori)-1)/2*picori

      pic[:,:,0]=B
      pic[:,:,-1]=C

      ee=round(ep[u],3)
      plt.imshow(pic)
      count=count+1


    plt.savefig(all_path+casenum+"/"+str(j)+"notitle.pdf")


class NeuralNetwork_minimum(nn.Module):
    def __init__(self):
        super(NeuralNetwork_minimum, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_tanh_stack = nn.Sequential(
            nn.Linear(2,32),
            nn.Tanh(),
            nn.Linear(32, 32),
            nn.Tanh(),
            nn.Linear(32, 32),
            nn.Tanh(),
            nn.Linear(32, 32),
            nn.Tanh(),
            nn.Linear(32, 1)
        )

    def forward(self, s):
        phi_pred = self.linear_tanh_stack(s)
        return phi_pred



class NeuralNetwork(nn.Module):
    def __init__(self,p1,m1):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_tanh_stack = nn.Sequential(
            nn.Linear(3,100),
            nn.Tanh(),
            nn.Linear(100, 100),
            nn.Tanh(),
            nn.Linear(100, 100),
            nn.Tanh(),
            nn.Linear(100, 100),
            nn.Tanh(),
            nn.Linear(100, 1)
        )
        self.p1=p1
        self.m1=m1


    def forward(self, s):
        x_pred = self.linear_tanh_stack(s)
        ss=s[:,0]
        ss=ss.reshape(-1,1)
        xx=s[:,1:3]

        out=ss*(1-ss)*x_pred + (1-ss)*self.p1(xx) + ss*self.m1(xx)

        return out



def train(model):
    loss_batch=[]
    plt_batch=[]
    lg_batch=[]

    if load_model:
        model.load_state_dict(torch.load(model_Parameters_name))

    for i in range(batches):
        if i<5000:
            learning_rate=1e-3
        else:
            learning_rate=1e-4


        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        if i<2000:
          alpha1=10
          alpha4=1
          alpha3=0.001
          alpha5=1

        else:
          alpha1=0
          alpha4=1
          alpha3=0.001
          alpha5=0.01


        datanum=50
        s_np = np.random.uniform(0.001,0.999,(datanum,1))
        x_np = np.random.uniform(0.001,0.999,(datanum,2))

        xxind=np.linspace(0,datanum-1,datanum)
        s_int_mesh_np,xxind_mesh_np=np.meshgrid(s_np,xxind)

        s_int_np=s_int_mesh_np.reshape((-1,1))
        xxind_np=xxind_mesh_np.reshape(-1)

        x_int_np=x_np[xxind_np.astype(np.int32),:]

        s_int_tensor=Variable(torch.from_numpy(s_int_np),requires_grad=True).to(device)
        x_int_tensor1=Variable(torch.from_numpy(x_int_np),requires_grad=True).to(device)

        x_all=[]
        for j in range(x_dim):
          x_all.append(x_int_tensor1[:,j:j+1])

        x_int_tensor=torch.cat(x_all,1)

        sx=torch.cat((s_int_tensor,x_int_tensor),1)

        phi_int_tensor=model(sx)


        gradx_all=[]
        abs_gradx_phi_2=0.0
        for j in range(x_dim):
          gradx_int = torch.autograd.grad(outputs=phi_int_tensor,inputs=x_all[j],grad_outputs=torch.ones_like(s_int_tensor),create_graph=True)[0]
          gradx_all.append(gradx_int)
          abs_gradx_phi_2=abs_gradx_phi_2+gradx_int**2

        V_int = kappa**2/2*abs_gradx_phi_2+(phi_int_tensor**2-1)**2/4

        V_int_mesh=V_int.reshape((datanum,datanum))
        F_aint=torch.mean(V_int_mesh,0)

        loss_1=1/beta*torch.log(torch.mean(torch.exp(beta*F_aint)))

        grads_int = torch.autograd.grad(outputs=phi_int_tensor,inputs=s_int_tensor,grad_outputs=torch.ones_like(s_int_tensor),create_graph=True)[0]
        grads2_int = torch.autograd.grad(outputs=grads_int,inputs=s_int_tensor,grad_outputs=torch.ones_like(s_int_tensor),create_graph=True)[0]
        loss_3=torch.mean(torch.abs(grads_int*grads2_int))

        gradx2_int=0.0
        for j in range(x_dim):
          gradx2_j=torch.autograd.grad(outputs=gradx_all[j],inputs=x_all[j],grad_outputs=torch.ones_like(s_int_tensor),create_graph=True)[0]
          gradx2_int=gradx2_int+gradx2_j


        L_phi_mesh_pre=(kappa**2*gradx2_int-(phi_int_tensor**3-phi_int_tensor))**2
        L_phi_mesh=L_phi_mesh_pre.reshape((datanum,datanum))
        L_int=torch.mean(L_phi_mesh,0)

        phi_s_mesh=grads_int.reshape((datanum,datanum))
        phi_int=torch.mean(phi_s_mesh**2,0)

        loss_4=torch.mean(torch.sqrt(L_int)*torch.sqrt(phi_int))

        s_b = np.random.uniform(0.001,0.999,(100,1))
        s_b_tensor=Variable(torch.from_numpy(s_b),requires_grad=True).to(device)

        xlu=np.random.uniform(0.001,0.999,(100,2))
        xlu[:,0]=0.0
        xlu_tensor=Variable(torch.from_numpy(xlu),requires_grad=True).to(device)


        xru=np.random.uniform(0.001,0.999,(100,2))
        xru[:,0]=1.0
        xru_tensor=Variable(torch.from_numpy(xru),requires_grad=True).to(device)


        xut=np.random.uniform(0.001,0.999,(100,2))
        xut[:,1]=1.0
        xut_tensor=Variable(torch.from_numpy(xut),requires_grad=True).to(device)

        xub=np.random.uniform(0.001,0.999,(100,2))
        xub[:,1]=0.0
        xub_tensor=Variable(torch.from_numpy(xub),requires_grad=True).to(device)


        sx_lu=torch.cat((s_b_tensor,xlu_tensor),1)
        sx_ru=torch.cat((s_b_tensor,xru_tensor),1)
        sx_ut=torch.cat((s_b_tensor,xut_tensor),1)
        sx_ub=torch.cat((s_b_tensor,xub_tensor),1)


        phi_lu=model(sx_lu)
        phi_ru=model(sx_ru)
        phi_ut=model(sx_ut)
        phi_ub=model(sx_ub)

        phi_p1_np=1.0*np.ones((100))
        phi_m1_np=-1.0*np.ones((100))

        phi_p1=Variable(torch.from_numpy(phi_p1_np),requires_grad=True).to(device)
        phi_m1=Variable(torch.from_numpy(phi_m1_np),requires_grad=True).to(device)

        loss_bound=torch.mean((phi_lu.reshape(-1)-phi_p1)**2)+torch.mean((phi_ru.reshape(-1)-phi_p1)**2)+torch.mean((phi_ut.reshape(-1)-phi_m1)**2)+torch.mean((phi_ub.reshape(-1)-phi_m1)**2)



        loss = alpha1*loss_1 +alpha4*loss_4+alpha3*loss_3+alpha5*loss_bound

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i) % 50 == 0:
            ss = np.linspace(0.001,0.999,dn,endpoint=True)
            x1 = np.linspace(0.001,0.999,dn,endpoint=True)
            x2 = np.linspace(0.001,0.999,dn,endpoint=True)

            xx1,xx2=np.meshgrid(x1,x2)
            xx1=xx1.reshape((-1,1))
            xx2=xx2.reshape((-1,1))

            xx=np.hstack((xx1,xx2))

            xxind=np.linspace(0,dn**2-1,dn**2)

            ss_int_mesh_np,xxind_int_mesh_np=np.meshgrid(ss,xxind)

            ss_int_np=ss_int_mesh_np.reshape((-1,1))
            xxind_np=xxind_int_mesh_np.reshape(-1)

            xx_int_np=xx[xxind_np.astype(np.int32),:]

            ss_int_tensor=Variable(torch.from_numpy(ss_int_np),requires_grad=True).to(device)
            xx_int_tensor1=Variable(torch.from_numpy(xx_int_np),requires_grad=True).to(device)

            xx_all=[]
            for j in range(x_dim):
              xx_all.append(xx_int_tensor1[:,j:j+1])

            xx_int_tensor=torch.cat(xx_all,1)

            ssxx=torch.cat((ss_int_tensor,xx_int_tensor),1)

            phi=model(ssxx)

            gradxx_all=[]
            abs_gradxx_phi_2=0.0
            for j in range(x_dim):
              gradxx_int = torch.autograd.grad(outputs=phi,inputs=xx_all[j],grad_outputs=torch.ones_like(ss_int_tensor),create_graph=True)[0]
              gradxx_all.append(gradxx_int)
              abs_gradxx_phi_2=abs_gradxx_phi_2+gradxx_int**2


            gradxx2_int=0.0
            for j in range(x_dim):
              gradxx2_j=torch.autograd.grad(outputs=gradxx_all[j],inputs=xx_all[j],grad_outputs=torch.ones_like(ss_int_tensor),create_graph=True)[0]
              gradxx2_int=gradxx2_int+gradxx2_j


            L_phi_mesh_pre=(kappa**2*gradxx2_int-(phi**3-phi))**2
            L_phi_mesh=L_phi_mesh_pre.reshape((dn**2,dn))
            L_int=torch.mean(L_phi_mesh,0)

            gradss_int = torch.autograd.grad(outputs=phi,inputs=ss_int_tensor,grad_outputs=torch.ones_like(ss_int_tensor),create_graph=True)[0]

            phi_ss_mesh=gradss_int.reshape((dn**2,dn))

            phi_int=torch.mean(phi_ss_mesh**2,0)

            lg=torch.mean(torch.sqrt(L_int)*torch.sqrt(phi_int))

            lg_batch.append(lg.item())

            loss, batch = alpha1*loss_1.item() + alpha4*loss_4.item() + alpha3*loss_3.item()+alpha5*loss_bound.item(), i
            print(f'batches: {batch+1}')
            print(f'loss1: {alpha1*loss_1} loss4: {alpha4*loss_4} loss3: {alpha3*loss_3} loss_bound: {alpha5*loss_bound}')

            loss_batch.append(loss)
            plt_batch.append(i+1)
            plt.figure()

        if (i) % 500 == 0:

            V_int = kappa**2/2*abs_gradxx_phi_2+(phi**2-1)**2/4

            V_int_mesh=V_int.reshape((dn**2,dn))
            F_aint=torch.mean(V_int_mesh,0)
            energy_plot=F_aint.cpu().detach().numpy()

            fig_cos(plt_batch,lg_batch,i)
            fig_loss_batch(plt_batch,loss_batch)
            fig_countour(xx_int_np,phi.cpu().detach().numpy(),energy_plot,i)
            plt.show()


            torch.save(model.state_dict(), model_name)
            print("Saved PyTorch Model State to " +str(model_name))

    output = open(path_lg, 'wb')
    pickle.dump(lg_batch,output)
    output.close()

    return




if __name__=='__main__':

    net_p1=NeuralNetwork_minimum().to(device)
    net_p1=net_p1.double()
    net_p1.load_state_dict(torch.load(path_p1))

    for p in net_p1.parameters():
        p.requires_grad=False


    net_m1=NeuralNetwork_minimum().to(device)
    net_m1=net_m1.double()
    net_m1.load_state_dict(torch.load(path_m1))


    for p in net_m1.parameters():
        p.requires_grad=False


    mkdir(all_path+casenum+"/")


    model = NeuralNetwork(net_p1,net_m1).to(device)
    model = model.double()

    #model = train_pre(model)
    train(model)



In [ ]:
from google.colab import drive
drive.mount('/content/drive')